<a href="https://colab.research.google.com/github/i-am-adithyavardhan/MiniProject_3rdYear/blob/main/bertCnnCombinedHasoc.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 58.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 30.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 101.7 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
from torch.autograd import Variable
import torch.nn.functional as functional
import matplotlib.pyplot as plt
from transformers import BertForSequenceClassification, AdamW, BertConfig
import gc
from transformers import BertModel
from sklearn.metrics import roc_auc_score,f1_score
import time
import datetime

In [ ]:
torch.manual_seed(0)
use_cuda = torch.cuda.is_available()
device = torch.device('cuda' if use_cuda else 'cpu')
if use_cuda:
    torch.cuda.manual_seed(0)
    
print("Using GPU: {}".format(use_cuda))

Using GPU: True


In [ ]:
train = pd.read_csv('/content/drive/MyDrive/HasocDataset/hindi_2021.csv')
train.head

<bound method NDFrame.head of       Unnamed: 0                       _id            tweet_id  \
0            998  60c5d7495659ea5e55df0b7b   hi_hasoc_2021_998   
1           4049  60c5d7495659ea5e55df1b73  hi_hasoc_2021_4049   
2           1757  60c5d7495659ea5e55df0d1b  hi_hasoc_2021_1757   
3           5175  60c5d7495659ea5e55df0e45  hi_hasoc_2021_5178   
4           1825  60c5d7495659ea5e55df0ee1  hi_hasoc_2021_1825   
...          ...                       ...                 ...   
4589        2481  60c5d7495659ea5e55df13ff  hi_hasoc_2021_2481   
4590        3711  60c5d7495659ea5e55df1b13  hi_hasoc_2021_3711   
4591        3113  60c5d7495659ea5e55df1678  hi_hasoc_2021_3113   
4592         824  60c5d7495659ea5e55df091b   hi_hasoc_2021_824   
4593        4309  60c5d7495659ea5e55df06c9  hi_hasoc_2021_4310   

                                                   text task_1 task_2  
0     @rssurjewala #Hindus DYING #HindusLivesMatter ...    NOT   NONE  
1     सब लोग इतने पैसे डोनेट ही क

In [ ]:
train = train.drop('task_2',axis=1)
train.columns = ['ID','TwitterID','HASOCID','Text','Type']
train = train.drop(['ID','TwitterID','HASOCID'],axis=1)

In [ ]:
train.rename(columns = {'Text':'tweet','Type':'label'}, inplace = True)
train.head

<bound method NDFrame.head of                                                   tweet label
0     @rssurjewala #Hindus DYING #HindusLivesMatter ...   NOT
1     सब लोग इतने पैसे डोनेट ही कर रहे हैं फिर भी आम...   NOT
2     शेर-ए- सिवान शहाबुद्दीन साहब से रिश्ता क्या  ل...   NOT
3            @AskAnshul आसमानी किताब के नाजायज औलाद है।   HOF
4     @Shikha0222 इसे कहते हैं दोगला पंती जिस सपा की...   NOT
...                                                 ...   ...
4589  सिवान के पुर्व सांसद मरहुम मो सहाबुदीन के रहस्...   NOT
4590  वर्ष 1989-1990 के दौरान कश्मीर के हिंदुओं के स...   NOT
4591  #किसानों_का_मोदी_को_धोबीपटका #ResignPMmodi htt...   NOT
4592  @Ravishk356 @newslaundry @abhisar_sharma मोदी ...   NOT
4593  बिहार के लोगों इन झूठे मक्कार राजनेताओं का चहर...   HOF

[4594 rows x 2 columns]>

In [ ]:
train["label"] = train["label"].map({"HOF": 0, "NOT": 1})
train.head

<bound method NDFrame.head of                                                   tweet  label
0     @rssurjewala #Hindus DYING #HindusLivesMatter ...      1
1     सब लोग इतने पैसे डोनेट ही कर रहे हैं फिर भी आम...      1
2     शेर-ए- सिवान शहाबुद्दीन साहब से रिश्ता क्या  ل...      1
3            @AskAnshul आसमानी किताब के नाजायज औलाद है।      0
4     @Shikha0222 इसे कहते हैं दोगला पंती जिस सपा की...      1
...                                                 ...    ...
4589  सिवान के पुर्व सांसद मरहुम मो सहाबुदीन के रहस्...      1
4590  वर्ष 1989-1990 के दौरान कश्मीर के हिंदुओं के स...      1
4591  #किसानों_का_मोदी_को_धोबीपटका #ResignPMmodi htt...      1
4592  @Ravishk356 @newslaundry @abhisar_sharma मोदी ...      1
4593  बिहार के लोगों इन झूठे मक्कार राजनेताओं का चहर...      0

[4594 rows x 2 columns]>

In [ ]:
from transformers import BertTokenizer
print('Loading BERT tokenizer...')
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)

Loading BERT tokenizer...


In [ ]:
tweets = train.tweet.values
labels = train.label.values

In [ ]:
import re
from string import punctuation
def preprocess(data):
    #remove url and hashtag
    for i in range(data.shape[0]):
        text=data[i].lower()
        text1=''.join([word+" " for word in text.split()])
        data[i]=text1
    giant_url_regex = ('http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|'
        '[!*,]|(?:%[0-9a-fA-F][0-9a-fA-F]))+')
    mention_regex = '@[\w\-]+'
    hashtag_regex = '#[\w\-]+'
    space_pattern = '\s+'

    for i in range(data.shape[0]):
        text_string = data[i]
        parsed_text = re.sub(hashtag_regex, '', text_string)
        parsed_text = re.sub(giant_url_regex, '', parsed_text)
        parsed_text = re.sub(mention_regex, '', parsed_text) 
        #remove punctuation
        parsed_text = re.sub(r"[{}]+".format(punctuation), '', parsed_text) 
        parsed_text = re.sub(space_pattern, ' ', parsed_text)
        data[i] = parsed_text
    return data
tweets = preprocess(tweets)
print(tweets)

[' dying liars i warned 2 yrs back banned by sardar patel ignored ेरनहींभेड़ियापाललियाभक्तो '
 'सब लोग इतने पैसे डोनेट ही कर रहे हैं फिर भी आम आदमी को सिलेंडर और कन्सेंट्रेटर ख़ुद से ही ख़रीदना पड़ रहा है तो पैसे कहाँ जा रहे हैं बीयर्ड ऑयल में '
 'शेरए सिवान शहाबुद्दीन साहब से रिश्ता क्या لا إله إلا الله محمد رسول الله '
 ... 'िसानोंकामोदीकोधोबीपटका ' ' मोदी तुम इस्तीफा दो हम तुम्हारे साथ है '
 'बिहार के लोगों इन झूठे मक्कार राजनेताओं का चहरा हमेशा याद रखना जब तुम्हारे सामने ये मक्कार वोटों की भीख मांगने आएं तो इन मक्कारों से पूछना कि जिस समय अपने बाप के जनाजे के लिए दरदर की ठोकरें खा रहा था उस समय तुम्हारी ये क़ौम से हमदर्दी कहां थी ']


In [ ]:
# Print the original sentence.
print(' Original: ', tweets[0])

# Print the sentence split into tokens.
print('Tokenized: ', tokenizer.tokenize(tweets[0]))

# Print the sentence mapped to token ids.
print('Token IDs: ', tokenizer.convert_tokens_to_ids(tokenizer.tokenize(tweets[0])))

 Original:   dying liars i warned 2 yrs back banned by sardar patel ignored ेरनहींभेड़ियापाललियाभक्तो 
Tokenized:  ['dying', 'liar', '##s', 'i', 'warned', '2', 'y', '##rs', 'back', 'banned', 'by', 'sar', '##dar', 'patel', 'ignored', 'र', '##न', '##ह', '##ी', '##भ', '##ड', '##ि', '##य', '##ा', '##प', '##ा', '##ल', '##ल', '##ि', '##य', '##ा', '##भ', '##क', '##त', '##ो']
Token IDs:  [5996, 16374, 2015, 1045, 7420, 1016, 1061, 2869, 2067, 7917, 2011, 18906, 7662, 20455, 6439, 1333, 29863, 29875, 29878, 29866, 29857, 29877, 29868, 29876, 29864, 29876, 29870, 29870, 29877, 29868, 29876, 29866, 29851, 29859, 29879]


In [ ]:
max_len = 0
ind = [100,200,300,400,500,512]
for i in ind:
  count = 0
  for tweet in tweets:
      max_len = max(max_len, len(tweet))
      if len(tweet)>i:
        count+=1
  print("Count of sentence length over {} is: ".format(i), count)
print('Max sentence length: ', max_len)

Count of sentence length over 100 is:  2255
Count of sentence length over 200 is:  629
Count of sentence length over 300 is:  0
Count of sentence length over 400 is:  0
Count of sentence length over 500 is:  0
Count of sentence length over 512 is:  0
Max sentence length:  282


In [ ]:
input_ids = []
attention_masks = []
for tweet in tweets:
    encoded_dict = tokenizer.encode_plus(
                        tweet,                   
                        add_special_tokens = True, 
                        max_length = 512,           
                        pad_to_max_length = True,
                        return_attention_mask = True,   
                        return_tensors = 'pt',     # Return pytorch tensors.
                   )
    input_ids.append(encoded_dict['input_ids'])
    
    attention_masks.append(encoded_dict['attention_mask'])
# Convert the lists into tensors.
input_ids = torch.cat(input_ids, dim=0)
attention_masks = torch.cat(attention_masks, dim=0)
labels = torch.tensor(labels)

# Print sentence 0, now as a list of IDs.
print('Original: ', tweets[0])
print('Token IDs:', input_ids[0])

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2354: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


Original:   dying liars i warned 2 yrs back banned by sardar patel ignored ेरनहींभेड़ियापाललियाभक्तो 
Token IDs: tensor([  101,  5996, 16374,  2015,  1045,  7420,  1016,  1061,  2869,  2067,
         7917,  2011, 18906,  7662, 20455,  6439,  1333, 29863, 29875, 29878,
        29866, 29857, 29877, 29868, 29876, 29864, 29876, 29870, 29870, 29877,
        29868, 29876, 29866, 29851, 29859, 29879,   102,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0, 

In [ ]:
from torch.utils.data import TensorDataset, random_split

dataset = TensorDataset(input_ids, attention_masks, labels)

train_size = int(0.9 * len(dataset))
val_size = len(dataset) - train_size

train_dataset, val_dataset = random_split(dataset, [train_size, val_size],generator=torch.Generator().manual_seed(42))
# df_train, df_val, df_test = np.split(df.sample(frac=1, random_state=42), 
#                                      [int(.8*len(df)), int(.9*len(df))])

print('{:>5,} training samples'.format(train_size))
print('{:>5,} validation samples'.format(val_size))

4,134 training samples
  460 validation samples


In [ ]:
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler

batch_size = 16

train_dataloader = DataLoader(
            train_dataset,  
            shuffle = True,
            batch_size = batch_size 
        )

validation_dataloader = DataLoader(
            val_dataset,
            shuffle = False,
            batch_size = batch_size 
        )
     

In [ ]:
def format_time(elapsed):
    elapsed_rounded = int(round((elapsed)))
    return str(datetime.timedelta(seconds=elapsed_rounded))

In [ ]:
from transformers import BertForSequenceClassification, AdamW, BertConfig
model = BertForSequenceClassification.from_pretrained(
    'bert-base-multilingual-cased',
    num_labels = 2, 
    output_attentions = False, 
    output_hidden_states = False,
)
model.to(device)

Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model ch

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(119547, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12

In [ ]:
optimizer = AdamW(model.parameters(),
                  lr = 5e-5, 
                  eps = 1e-8 
                )
epochs = 4
criterion = nn.CrossEntropyLoss()

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [ ]:
import random
import numpy as np

seed_val = 42

random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

training_stats = []
total_t0 = time.time()
best_accuracy = 0
for epoch_i in range(0, epochs):
    #Training
    print("")
    print('Epoch {:} / {:}'.format(epoch_i + 1, epochs))
    print('Training...')

    t0 = time.time()
    total_train_loss = 0
    total_train_accuracy = 0
    model.train()
    for step, batch in enumerate(train_dataloader):

        input_ids = batch[0].to(device)
        input_mask = batch[1].to(device)
        labels = batch[2].to(device)

        model.zero_grad()        
        out = model(input_ids, token_type_ids=None, attention_mask=input_mask, labels=labels)
        loss = out[0]
        logits = out[1]
  
        total_train_loss += loss.item()
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        optimizer.step()

        pred = torch.argmax(logits, dim = 1)
        total_train_accuracy +=  torch.sum(pred == labels).item()
        
    avg_train_accuracy = total_train_accuracy / len(train_dataloader.dataset)
    avg_train_loss = total_train_loss / len(train_dataloader.dataset)            
    print("  Accuracy: {}".format(avg_train_accuracy))
    print("  Training loss: {}".format(avg_train_loss))
        

    # Validation
    print("")
    print("Validation...")
    model.eval()
    total_eval_accuracy = 0
    total_eval_loss = 0
    y_true = []
    y_pred = []
    for batch in validation_dataloader:
        input_ids = batch[0].to(device)
        input_mask = batch[1].to(device)
        labels = batch[2].to(device)
        
        with torch.no_grad():        
            out = model(input_ids, token_type_ids=None, attention_mask=input_mask,labels=labels)
            loss = out[0]
            logits = out[1]

        total_eval_loss += loss.item()
        pred = torch.argmax(logits, dim = 1)
        total_eval_accuracy += torch.sum(pred == labels).item()
        y_true.append(labels.flatten())
        y_pred.append(pred.flatten())
        
    avg_val_accuracy = total_eval_accuracy / len(validation_dataloader.dataset)
    print("  Accuracy: {}".format(avg_val_accuracy))
    avg_val_loss = total_eval_loss / len(validation_dataloader.dataset)
    print("  Validation loss: {}".format(avg_val_loss))
    training_time = format_time(time.time() - t0)
    print()
    
    y_true = torch.cat(y_true).tolist()
    y_pred = torch.cat(y_pred).tolist()
    print("This epoch took: {:}".format(training_time))
    print('roc_auc score: ', roc_auc_score(y_true,y_pred))
    print('F1 score:',f1_score(y_true, y_pred))
    print()

    training_stats.append(
        {
            'epoch': epoch_i + 1,
            'Train Accur.': avg_train_accuracy,
            'Training Loss': avg_train_loss,
            'Valid. Loss': avg_val_loss,
            'Valid. Accur.': avg_val_accuracy,
            'Training Time': training_time,
        }
    )
    print()

    if avg_val_accuracy > best_accuracy:
        best_accuracy = avg_val_accuracy
        best_model = model

print()
print("="*10)
print("Summary")
print("Total time {:} (h:mm:ss)".format(format_time(time.time()-total_t0)))

In [ ]:
def evaluate(model, test_data):

    test = Dataset(test_data)

    test_dataloader = torch.utils.data.DataLoader(test, batch_size=2)

    use_cuda = torch.cuda.is_available()
    device = torch.device("cuda" if use_cuda else "cpu")

    if use_cuda:

        model = model.cuda()

    total_acc_test = 0
    with torch.no_grad():

        for test_input, test_label in test_dataloader:

              test_label = test_label.to(device)
              mask = test_input['attention_mask'].to(device)
              input_id = test_input['input_ids'].squeeze(1).to(device)

              output = model(input_id, mask)

              acc = (output.argmax(dim=1) == test_label).sum().item()
              total_acc_test += acc
    
    print(f'Test Accuracy: {total_acc_test / len(test_data): .3f}')
    
evaluate(model, df_test)

In [ ]:
PATH1 = "/content/drive/MyDrive/HasocDataset/Models/hindi_model_bert.pt"
the_best_model = torch.load(PATH1,map_location='cpu')

In [ ]:
class BertCNNClassifier(nn.Module):
    def __init__(self, tuned_model, embed_num = 512, embed_dim = 768, dropout=0.1, kernel_num=3, kernel_sizes=[1,2], num_labels=2):
        super().__init__()
        self.num_labels = num_labels
        self.embed_num = embed_num
        self.embed_dim = embed_dim
        self.dropout = dropout
        self.kernel_num = kernel_num
        self.kernel_sizes = kernel_sizes
        self.softmax = nn.functional.softmax

        self.bert = tuned_model.bert
        self.convs = nn.ModuleList([nn.Conv2d(1, self.kernel_num, (k, self.embed_dim)) for k in self.kernel_sizes])
        self.dropout = nn.Dropout(self.dropout)
        self.classifier = nn.Linear(len(self.kernel_sizes)*self.kernel_num, self.num_labels)
        
    def forward(self, input_ids, attention_mask, token_type_ids = None):
        output = self.bert(input_ids = input_ids, attention_mask = attention_mask, token_type_ids = token_type_ids) #16,512,768
        output = output[0].unsqueeze(1) #16,1,512,768
        output = [nn.functional.relu(conv(output)).squeeze(3) for conv in self.convs] #16,3,508,1 => #16,3,508
        output = [nn.functional.max_pool1d(i, i.size(2)).squeeze(2) for i in output] #=> 16,3
        output = torch.cat(output, 1)
        output = self.dropout(output)
        logits = self.classifier(output)
        return self.softmax(logits, 1)

In [ ]:
# Initializing model
model2 = BertCNNClassifier(the_best_model).cuda()
# set parameters
epochs = 4
learning_rate = 5e-5
optimizer = AdamW(model2.parameters(), lr = learning_rate)
criterion = nn.CrossEntropyLoss()

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [ ]:
import random
import numpy as np

seed_val = 42

random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

training_stats = []
total_t0 = time.time()
best_accuracy = 0
for epoch_i in range(0, epochs):
    #Training
    print("")
    print('Epoch {:} / {:}'.format(epoch_i + 1, epochs))
    print('Training...')

    t0 = time.time()
    total_train_loss = 0
    total_train_accuracy = 0
    model2.train()
    for step, batch in enumerate(train_dataloader):

        input_ids = batch[0].to(device)
        input_mask = batch[1].to(device)
        labels = batch[2].to(device)

        model2.zero_grad()        
        out = model2(input_ids = input_ids, attention_mask = input_mask, token_type_ids = None)
        loss = criterion(out, labels)
        total_train_loss += loss.item()

        loss.backward()
        torch.nn.utils.clip_grad_norm_(model2.parameters(), 1.0)
        optimizer.step()

        pred = torch.argmax(out, dim = 1)
        total_train_accuracy +=  torch.sum(pred == labels).item()

    avg_train_accuracy = total_train_accuracy / len(train_dataloader.dataset)
    avg_train_loss = total_train_loss / len(train_dataloader.dataset)            
    print("  Accuracy: {}".format(avg_train_accuracy))
    print("  Training loss: {}".format(avg_train_loss))
        
    # Validation
    print("")
    print("Validation...")
    model2.eval()
    total_eval_accuracy = 0
    total_eval_loss = 0
    y_true = []
    y_pred = []

    for batch in validation_dataloader:
        input_ids = batch[0].to(device)
        input_mask = batch[1].to(device)
        labels = batch[2].to(device)
        
        with torch.no_grad():        
            out = model2(input_ids = input_ids, attention_mask = input_mask, token_type_ids = None)
        loss = criterion(out, labels)
        total_eval_loss += loss.item()
        pred = torch.argmax(out, dim = 1)
        total_eval_accuracy += torch.sum(pred == labels).item()
        y_true.append(labels.flatten())
        y_pred.append(pred.flatten())
        
    avg_val_accuracy = total_eval_accuracy / len(validation_dataloader.dataset)
    print("  Accuracy: {}".format(avg_val_accuracy))
    avg_val_loss = total_eval_loss / len(validation_dataloader.dataset)
    print("  Validation loss: {}".format(avg_val_loss))
    training_time = format_time(time.time() - t0)
    print("  This epoch took: {:}".format(training_time))
    print()
    y_true = torch.cat(y_true).tolist()
    y_pred = torch.cat(y_pred).tolist()
    print('  roc_auc score: ', roc_auc_score(y_true,y_pred))
    print('  F1 score:',f1_score(y_true, y_pred))


    training_stats.append(
        {
            'epoch': epoch_i + 1,
            'Train Accur.': avg_train_accuracy,
            'Training Loss': avg_train_loss,
            'Valid. Loss': avg_val_loss,
            'Valid. Accur.': avg_val_accuracy,
            'Training Time': training_time,
        }
    )

    if avg_val_accuracy > best_accuracy:
        best_accuracy = avg_val_accuracy
        best_model = model2

print("===")
print("Summary")
print("Total time {:} (h:mm:ss)".format(format_time(time.time()-total_t0)))
print('best acc:',best_accuracy)


Epoch 1 / 4
Training...
  Accuracy: 0.6862602805999033
  Training loss: 0.040438274282963885

Validation...
  Accuracy: 0.7043478260869566
  Validation loss: 0.040325025371883226
  This epoch took: 0:06:07

  roc_auc score:  0.5
  F1 score: 0.826530612244898

Epoch 2 / 4
Training...
  Accuracy: 0.6862602805999033
  Training loss: 0.04044798583181572

Validation...
  Accuracy: 0.7043478260869566
  Validation loss: 0.040325025371883226
  This epoch took: 0:06:08

  roc_auc score:  0.5
  F1 score: 0.826530612244898

Epoch 3 / 4
Training...
  Accuracy: 0.6862602805999033
  Training loss: 0.04044313705019404

Validation...
  Accuracy: 0.7043478260869566
  Validation loss: 0.040325025371883226
  This epoch took: 0:06:08

  roc_auc score:  0.5
  F1 score: 0.826530612244898

Epoch 4 / 4
Training...
  Accuracy: 0.6862602805999033
  Training loss: 0.040427179413252426

Validation...
  Accuracy: 0.7043478260869566
  Validation loss: 0.040325025371883226
  This epoch took: 0:06:07

  roc_auc scor

In [ ]:
PATH2 = "/content/drive/MyDrive/HasocDataset/Models/hindi_bertCnn.pt"
torch.save(model2, PATH2)

In [ ]:
class BertLstmClassifier(nn.Module):
    def __init__(self, model_tune):
        super().__init__()
        self.bert = model_tune.bert
        self.lstm = nn.LSTM(input_size = 768, 
                            hidden_size = 768, 
                            num_layers = 1,
                            batch_first = True, #(batch,seq,feature)
                            bidirectional = True)
        self.classifier = nn.Linear(768 * 2, 2)
        self.softmax = nn.Softmax(dim = 1)

    def forward(self, input_ids, attention_mask, token_type_ids):
        bert_output = self.bert(input_ids = input_ids, attention_mask = attention_mask, token_type_ids = token_type_ids)
        out, _ = self.lstm(bert_output[0])
        logits = self.classifier(out[:, 1, :])
        return self.softmax(logits)

In [ ]:
# Initializing model
model3 = BertLstmClassifier(the_best_model).cuda()
# set parameters
epochs = 6
learning_rate = 5e-5
optimizer = AdamW(model3.parameters(), lr = learning_rate)
criterion = nn.CrossEntropyLoss()

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [ ]:
import random
import numpy as np

seed_val = 42

random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

training_stats = []
total_t0 = time.time()
best_accuracy = 0
for epoch_i in range(0, epochs):
    #Training
    print("")
    print('Epoch {:} / {:}'.format(epoch_i + 1, epochs))
    print('Training...')

    t0 = time.time()
    total_train_loss = 0
    total_train_accuracy = 0
    model3.train()
    for step, batch in enumerate(train_dataloader):

        input_ids = batch[0].to(device)
        input_mask = batch[1].to(device)
        labels = batch[2].to(device)

        model3.zero_grad()        
        out = model3(input_ids = input_ids, attention_mask = input_mask, token_type_ids = None)
        loss = criterion(out, labels)
        total_train_loss += loss.item()

        loss.backward()
        torch.nn.utils.clip_grad_norm_(model3.parameters(), 1.0)
        optimizer.step()

        pred = torch.argmax(out, dim = 1)
        total_train_accuracy +=  torch.sum(pred == labels).item()
        
    avg_train_accuracy = total_train_accuracy / len(train_dataloader.dataset)
    avg_train_loss = total_train_loss / len(train_dataloader.dataset)            
    print("  Accuracy: {}".format(avg_train_accuracy))
    print("  Training loss: {}".format(avg_train_loss))
        
    # Validation
    print("")
    print("Validation...")
    model3.eval()
    total_eval_accuracy = 0
    total_eval_loss = 0
    y_true = []
    y_pred = []

    for batch in validation_dataloader:
        input_ids = batch[0].to(device)
        input_mask = batch[1].to(device)
        labels = batch[2].to(device)
        
        with torch.no_grad():        
            out = model3(input_ids = input_ids, attention_mask = input_mask, token_type_ids = None)
        loss = criterion(out, labels)
        total_eval_loss += loss.item()
        pred = torch.argmax(out, dim = 1)
        total_eval_accuracy += torch.sum(pred == labels).item()
        y_true.append(labels.flatten())
        y_pred.append(pred.flatten())
        
    avg_val_accuracy = total_eval_accuracy / len(validation_dataloader.dataset)
    print("  Accuracy: {}".format(avg_val_accuracy))
    avg_val_loss = total_eval_loss / len(validation_dataloader.dataset)
    print("  Validation loss: {}".format(avg_val_loss))
    training_time = format_time(time.time() - t0)
    print("  This epoch took: {:}".format(training_time))
    print()
    y_true = torch.cat(y_true).tolist()
    y_pred = torch.cat(y_pred).tolist()
    print('  roc_auc score: ', roc_auc_score(y_true,y_pred))
    print('  F1 score:',f1_score(y_true, y_pred))

    training_stats.append(
      {
          'epoch': epoch_i + 1,
          'Train Accur.': avg_train_accuracy,
          'Training Loss': avg_train_loss,
          'Valid. Loss': avg_val_loss,
          'Valid. Accur.': avg_val_accuracy,
          'Training Time': training_time,
      }
    )

    if avg_val_accuracy > best_accuracy:
        best_accuracy = avg_val_accuracy
        best_model = model3

print("===")
print("Summary")
print("Total time {:} (h:mm:ss)".format(format_time(time.time()-total_t0)))
print('best acc:',best_accuracy)



Epoch 1 / 6
Training...
  Accuracy: 0.6857764876632801
  Training loss: 0.03908381873158712

Validation...
  Accuracy: 0.7043478260869566
  Validation loss: 0.03823878441167914
  This epoch took: 0:07:30

  roc_auc score:  0.5
  F1 score: 0.826530612244898

Epoch 2 / 6
Training...
  Accuracy: 0.6862602805999033
  Training loss: 0.03899961157760934

Validation...
  Accuracy: 0.7043478260869566
  Validation loss: 0.03823856458715771
  This epoch took: 0:07:32

  roc_auc score:  0.5
  F1 score: 0.826530612244898

Epoch 3 / 6
Training...
  Accuracy: 0.6862602805999033
  Training loss: 0.039062443646648744

Validation...
  Accuracy: 0.7043478260869566
  Validation loss: 0.03821486526209375
  This epoch took: 0:07:31

  roc_auc score:  0.5
  F1 score: 0.826530612244898

Epoch 4 / 6
Training...
  Accuracy: 0.6862602805999033
  Training loss: 0.03895658204880785

Validation...
  Accuracy: 0.7043478260869566
  Validation loss: 0.038223072562528694
  This epoch took: 0:07:33

  roc_auc score:  

In [ ]:
PATH4 = "/content/drive/MyDrive/Colab Notebooks/'hind_bertlstm.pt'"
torch.save(model3, PATH4)